## CHAPTER13.RDD 고급

- Key-value 형태의 RDD를 중심으로 RDD 고급 연산
- 사용자 정의 파티셔닝 => 클러스터에 데이터가 배치되는 방식을 정확히 제어 가능 / 개별 파티션 다루기
- RDD 조인

In [1]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

In [2]:
print(myCollection)

['Spark', 'The', 'Definitive', 'Guide', ':', 'Big', 'Data', 'Processing', 'Made', 'Simple']


In [3]:
words = spark.sparkContext.parallelize(myCollection, 2)

### 13.1 키-값 형태의 기초(키-값 형태의 RDD)

- 다양한 메서드
- <연산명>ByKey
- 메서드 명에 ByKey가 있다면 PariRDD 타입만 사용 가능
- 간단하게 pairRDD 타입을 만드는 방법(예시): RDD에 맵 연산을 수행해 키-값 구조로 만드는 것 -> RDD 레코드에 두 개의 값 생성

### 13.2.1 countByKey

- 각 키의 아이템 수를 구하고 로컬 맵으로 결과를 수집
- 스칼라/자바 -> 제한 시간(timeout)과 신뢰도를 인수 지정하여 근사치를 구할 수 있음

### 13.2.2 집계 연산 구현 방식 이해하기
- PairRDD 구현 방식 -> 잡의 안정성을 위해 중요
<예시>groupBy 와 reduce 함수 비교
- 공통점: 키를 기준으로 비교

- groupByKey
    - 익스큐터에서 함수를 적용하기 전 해당 키와 관련된 "모든 값"을 메모리로 읽어들여야 함.
    - 심각하게 치유쳐진 키가 이다면, 일부 파티션이 엄청난 양의 값을 가질 수 있어 OutOfMemoryError 발생
    - 각 키에 대한 값의 크기가 일정하고 익스큐터에 할당된 메모리에서 처리 가능할 정도의 크기일 경우 groupByKey 메서드 사용을 권장
- reduceByKey
    - 각 파티션에서 리듀스 작업을 수행하기에 훨씬 안정적이며, 모든 값을 메모리에 유지할 필요 없음
    - 최종 리듀스 과정을 제외한 모든 작업은 개별 워커에서 처리하기 때문에, 연산 중 셔플 발생X
    - 키별 그룹 RDD를 반환.
    - RDD의 개별 요소들은 정렬되어있지 않음.
        => 작업부하를 줄이려는 경우에 적합. 결과 순서가 중요한 경우에는 부적합
        
### 13.2.3 기타 집게 메서드

- 다양한 고급 집계 함수 존재
- 사용자 워크로드에 따라 세부 구현 방법 차이 있을 수 있음
- 고급 집계 함수 사용 시, 클러스터 노드에서 수행하는 집계를 구체적이고 세밀하게 제어 가능

#### aggregate
- null값이나 시작값이 필요
- 두 가지 함수를 파라미터로 사용
    - 첫 번째 함수는 파티션 내에서 수행. 시작값 사용
    - 두 번째 함수는 모든 파티션에 걸쳐 수행. 시작갑 사용
- 드라이버에서 최종 집계 수행 -> 성능에 약간의 영향이 있음
(ex)익스큐터의 결과가 너무 크면 OutOfMemory 발생으로 드라이버가 비정상적으로 종료
- treeAggregate
    - 드라이버에서 최종 집계를 수행하기 전에 익스큐터끼리 트리를 형성해 집계 처리의 일부 하위 과정을 '푸시 다운' 방식으로 먼저 수행
    - 집계 처리를 여러 단계로 구성하는 것은 드라이버의 메모리를 모두 서비하는 현상 방지에 도움
    
#### aggregateByKey

- aggregate 함수와 동일하지만 파티션 대신 키를 기준으로 연산을 수행

#### combineByKey

- 집계 함수 대신 컴바이너(combiner)를 사용
- 컴바이너(combiner): 키를 기준으로 연산을 수행하며 파라미터로 사용된 함수에 따라 값을 병합
- 여러 컴바이너의 결과값을 병합해 결과를 반환
- 사용자 정의 파티셔너를 사용해 출력 파티션 수를 지정할 수도 있음.

#### foldByKey

- 결합 함수와 항등원인 '제로값'을 이용해 각 키의 값을 병합
- 제록ㅂㅅ은 결과에 여러 번 사용될 수 있으나, 결과를 변경할 수는 없음

### 13.3 cogroup
- 스칼라는 최대 3개, 파이썬을 사용하는 경우 최대 2개의 키-값 형태의 RDD 그룹화 가능
- 각 키를 기준으로 값을 결합
- 출력 파티션 수, 클러스터에 데이터 분산 방식을 정확하게 제어하기 위해 사용자 정의 파티션 함수를 파라미터로 사용 가능
- 그룹화된 키를 '키'로, 키와 관련된 모든 값을 '값'으로 하는 키-값 형태의 배열을 결과로 반환

### 13.4 조인
- RDD는 구조적 API에서 알아본 것과 거의 동일한 조인 방식
- RDD를 사용하는 경우 사용자가 많은 부분에 관여해야 한다.
- 조인하기 위해서 두 개의 RDD가 기본적으로 필요
- 때에 따라 출력 파티션 수나 사용자 정의 파티션 함수를 파라미터로 사용

#### 13.4.1 내부 조인(INNER JOIN)
- 출력 파티션 수 설정 방법 참고(예제: outputPartitions = 10, 파라미터 지정)
- 다른 조인 함수 예제는 미제공이지만 모두 동일한 기본 형식을 따름
    - fullOuterJoin
    - leftOuterJoin
    - rightOuterJoin
    - cartesian <- 사용 시 주의주의. 조인 키를 사용하지 않기 때문에 출력 결과가 어마어마해질 수 있음
    
#### 13.4.2 zip
- 조인은 아니지만 두 개의 RDD를 결합하므로 조인이라 볼 수 있음
- zip 함수를 사용해 동일한 길이의 두 개의 RDD를 지퍼 잠그듯이 연결 가능하며 PairRDD를 생성
- 두 개의 RDD는 동일한 수의 요소와 동일한 수의 파티션을 가져야 한다.

### 13.5 파티션 제어하기
- RDD를 사용하면 데이터가 클러스터 전체에 물리적으로 정확히 분산되는 방식을 정의 할 수 있음
- 구조적 API와 가장 큰 차이점은 파티션 함수를 파라미터로 사용할 수 있다는 것
- 파티션 함수는 보통 사용자 지정 Partitioner를 의미.

#### 13.5.1 coalesce
- 파티션을 재분배할 때 발생하는 데이터 셔플을 방지하기 위해 동일한 워커에 존재하는 파티션을 합치는 메서드
<예제> 기존 두 개의 파티션으로 구성된 RDD를 데이터 셔플링 없이 하나의 파티션으로 합치는 것

#### 13.5.2 repartition
- 파티션 수를 늘리거나 줄일 수 있음
- 처리 시 노드 간의 셔플이 발생할 수 있음

#### 13.5.3 repartitionAndSortWithinPartitions
- 파티션을 재분배 할 수 있고, 재분배된 결과 파티션의 정렬 방식을 지정할 수 있음.
- 파티셔닝과 키 비교 모두 사용자가 지정 가능
- 관련 문서 참고

#### 13.5.4 사용자 정의 파티셔닝
- RDD를 사용하는 큰 이유 중 하나
- 저수준 API의 세부적인 구현 방식
- 잡이 성공적으로 동작되는지 여부에 상당한 영향.
- 대표적인 예시: 페이지 랭크(PageRank)
    - 사용자 정의 파티셔닝을 이용해 클러스터의 데이터 배치 구조를 제어하고 셔플을 회피
- 사용자정의 파티셔닝의 목표: 데이터 치우침(skew) 문제를 피하고자 클러스터 전체에 걸쳐 데이터를 균등하게 분배하는 것
- 구조적 API로 RDD를 얻고 사용자 정의 하티셔너를 적용한 다음 다시 DataFrame이나 Dataset으로 변환 -> 필요시에만 사용자 정의 파티셔닝을 사용할 수 있어 구조적 API와 RDD의 장점을 모두 활용 가능
- 사용자 정의 파티셔닝을 사용하기 위해서는 Partitioner를 확장한 클래스를 구현해야 함.
- 문제에 대해 업무 지식을 충분히 가지고 있는 경우에만 사용할 것을 권장.
- 단일 값이나 다수 값(다수 컬럼)을 파티셔닝 해야한다면 DataFrame API를 사용하는 것을 추천

- HashPartitioner(이산형)/RangePartitioner(연속형): RDD API에서 사용할 수 있는 내장형 파티셔너.(구조적API, RDD 모두 사용 가능)
- 키 치우침: 어떤 키가 다른 키에 비해 아주 많은 데이터를 가지는 현상
- 병렬성을 개선하고 실행 과정에서 OutOfMemoryError 방지 위해 키를 최대한 분할.

- 사용자 정의 키 분배 로직은 RDD 수준에서만 사용 가능.
- 사용자 정의 키 분배 로직은 임의의 로직을 사용해 물리적인 방식으로 클러스터에 데이터를 분배하는 강력한 방법.

### 13.6 사용자 정의 직렬화
- Kryo 직렬화
- 기본 직렬화 기능은 느릴 수 있음.
- 스파크는 Kryo 라이브러리(버전 2)를 사용해 더 빠르게 객체를 직렬화 할 수 있음
- 모든 직렬화 유형을 지원하지는 않는다.
- 프로그램에서 사용할 클래스를 사전에 등록해야 함
- 네트워크에 민감한 애플리케이션에서 사용할 것을 권장.
- 스파크 2.0.0 버전부터는 단순 데이터타입, 단순 데이터 타입의 배열, 문자열 데이터 타입의 RDD를 셔플링하면 내부적으로 Kryo 시리얼라이저를 사용
- Kryo에 사용자 정의 클래스를 등록하려면 registerKryoClasses 메서드를 사용

In [44]:
# words.countApproxDistinct(0.05)

In [4]:
words.map(lambda word:(word.lower(), 1))

PythonRDD[1] at RDD at PythonRDD.scala:53

In [5]:
keyword = words.keyBy(lambda word: word.lower()[0])

In [7]:
# keyword.mapValues(lambda word: word.upper()).collect() # python version conflict

In [9]:
# keyword.flatMapValues(lambda word: word.upper()).collect()

In [11]:
# keyword.keys().collect()
# keyword.values().collect()

In [105]:
# import random
# distinctChars = words.flatMap(lambda word: list(word.lower())).distinct()\
#     .collect()
# sampleMap = dict(map(lambda c: (c, random.random()), distinctChars))

# words.map(lambda word: (word.lower()[0], word))\
#     .sampleByKey(Trud, sampleMap, 6).collect()


In [81]:
def maxFunc(left, right):
    return max(left, right)
def addFunc(left, right):
    return left + right

In [82]:
nums = sc.parallelize(range(1,31),5)

In [84]:
# KVcharacters.countByKey()

In [86]:
# KVcharacters.grouopByKey().map(lambda row: (row[0], reduce(addFunc, row[1])))\
#     .collect()

In [88]:
# KVcharacters.reduceByKey(addFunc).collect()

In [90]:
# nums.aggregate(0, maxFunc, addFunc)

In [92]:
# depth = 3
# nums.treeAggregate(0, maxFunc, addFunc, depth)

In [94]:
# KVcharacters.aggregateByKey(0,addFunc,maxFunc).collect()

In [96]:
# def valToCombiner(value):
#     return [value]
# def mergeValuesFunc(vals, valToAppend):
#     vals.append(valToAppend)
#     return vals
# def mergecombinerFunc(vals1, vals2):
#     return vals1 + vals2
# outputPartitions = 6

# KVcharacters\
#     .combineByKey(
#         valToCombiner,
#         mergeValuesFunc,
#         mergeCombinerFunc,
#         outputPartitions)\
#     .collect()

In [69]:
# KVcharacters.foldByKey(0, addFunc).collect()

In [66]:
# import random

# distinctChars = words.flatMap(lambda word: word.lower()).distinct()

In [67]:
# keyedChars = distinctChars.map(lambda c: (c,random.random()))
# outputPartitions = 10

# KVcharacters.join(KeyedChars).count()
# KVcharacters.join(keyedChars, outputPartitions).count()

In [62]:
numRange = sc.parallelize(range(10),2)
# words.zip(numRange).collect()

In [56]:
words.coalesce(1).getNumPartitions()

1

In [58]:
# words.repartition(10)

In [49]:
# df = spark.read.option("header","true").option("inferSchema","true")\
#     .csv("../../assets/exercises/week13/retail-data/all/")

In [51]:
# rdd = df.coalesce(10).rdd

In [52]:
# df.printSchema()

In [55]:
# def partitionFunc(key):
#     import random
    
#     if key == 17850 or key == 12583:
#         return 0
#     else:
#         return random.randint(1,2)
    
# keyedRDD = rdd.keyBy(lambda row: row[6])

# keyedRDD\
#     .partitionBy(3, partitionFunc)\
#     .map(lambda x: x[0])\
#     .glom()\
#     .map(lambda x: len(set(x)))\
#     .take(5)